In [13]:
import pandas as pd 
import numpy as np 
import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import confusion_matrix

# Deep Neural Networks Laboratory

## Dataset

A Neural Network should always be built considering the dataset at hand.

In our case, we will continue our example with the Wine Dataset we built in the previous notebook.

In [14]:
class GenericDataset(Dataset):
    def __init__(self, targets_file, data_file, transform=None, target_transform=None):
        self.targets_file = pd.read_csv(targets_file)
        self.data_dir = pd.read_csv(data_file)
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.targets_file)

    def __getitem__(self, idx):
        return self.data_dir.iloc[idx].to_numpy(dtype=np.float32), self.targets_file.iloc[idx].item()

In [15]:
folder = os.path.join('data', 'iris')

In [16]:
WineDataset = GenericDataset(targets_file=os.path.join(folder, 'targets.csv'), data_file=os.path.join(folder, 'data.csv'))

In [17]:
train_size = int(0.9 * len(WineDataset))
test_size = len(WineDataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(WineDataset, [train_size, test_size])

In [18]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [19]:
next(iter(train_dataloader))

[tensor([[ 0.6722, -0.5858,  1.0436,  1.3121],
         [-1.2600, -0.1245, -1.3368, -1.1776],
         [ 1.5176, -0.1245,  1.2136,  1.1811],
         [ 0.5515, -1.7390,  0.3635,  0.1328],
         [ 0.6722, -0.3552,  0.3068,  0.1328],
         [-1.3807,  0.3367, -1.2234, -1.3086],
         [ 0.1892, -0.3552,  0.4202,  0.3948],
         [-0.8977,  1.7205, -1.2234, -1.3086],
         [ 0.3100, -0.3552,  0.5335,  0.2638],
         [ 0.4307,  0.7980,  0.9302,  1.4431],
         [-1.6223, -1.7390, -1.3935, -1.1776],
         [-1.5015,  0.3367, -1.3368, -1.3086],
         [-0.0523, -0.8164,  0.7602,  0.9190],
         [-0.1731, -0.5858,  0.4202,  0.1328],
         [-1.1392, -1.2777,  0.4202,  0.6569],
         [-1.0184, -2.4308, -0.1466, -0.2603],
         [ 1.2761,  0.1061,  0.7602,  1.4431],
         [-1.0184,  1.2592, -1.3368, -1.3086],
         [ 2.4837,  1.7205,  1.4970,  1.0500],
         [ 0.4307, -0.5858,  0.5902,  0.7880],
         [ 0.6722,  0.1061,  0.9869,  0.7880],
         [ 1.

## Neural Network

Pytorch is useful to build fully customizable Neural Networks.
Keep in mind these three things:

1. Our Neural Network will extend the class ```nn.Module```. 

2. The layers will be initialized inside ```__init__```.

3. The operations on the input data are defined in the ```forward``` method.

In [20]:
class FirstNeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(4, 10)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(10, 40)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(40, 20)
        self.relu3 = nn.ReLU()
        self.output_layer = nn.Linear(20, 3)  

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.output_layer(x)
        return x

In [21]:
model = FirstNeuralNetwork()

In [22]:
def train_loop(dataloader, model, loss_fn, optimizer):
  
  size = len(dataloader.dataset)
  for batch, (X, y) in enumerate(dataloader):

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if batch % 100 == 0:
      loss, current = loss.item(), (batch + 1) * len(X)
      print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
  size = len(dataloader.dataset)
  num_batches = len(dataloader)
  test_loss, correct = 0, 0

  with torch.no_grad():
    for X, y in dataloader:
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
learning_rate = 1e-1
batch_size = 32
epochs = 20

In [24]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 1.091637  [   32/  135]
Test Error: 
 Accuracy: 26.7%, Avg loss: 1.150203 

Epoch 2
-------------------------------
loss: 1.090529  [   32/  135]
Test Error: 
 Accuracy: 26.7%, Avg loss: 1.146206 

Epoch 3
-------------------------------
loss: 1.077711  [   32/  135]
Test Error: 
 Accuracy: 26.7%, Avg loss: 1.127584 

Epoch 4
-------------------------------
loss: 1.047767  [   32/  135]
Test Error: 
 Accuracy: 26.7%, Avg loss: 1.102759 

Epoch 5
-------------------------------
loss: 1.055685  [   32/  135]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.068471 

Epoch 6
-------------------------------
loss: 1.034009  [   32/  135]


Test Error: 
 Accuracy: 33.3%, Avg loss: 1.039135 

Epoch 7
-------------------------------
loss: 1.012503  [   32/  135]
Test Error: 
 Accuracy: 40.0%, Avg loss: 1.008058 

Epoch 8
-------------------------------
loss: 0.989487  [   32/  135]
Test Error: 
 Accuracy: 86.7%, Avg loss: 0.926916 

Epoch 9
-------------------------------
loss: 0.938347  [   32/  135]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.840304 

Epoch 10
-------------------------------
loss: 0.860558  [   32/  135]
Test Error: 
 Accuracy: 73.3%, Avg loss: 0.746502 

Epoch 11
-------------------------------
loss: 0.801235  [   32/  135]
Test Error: 
 Accuracy: 73.3%, Avg loss: 0.641173 

Epoch 12
-------------------------------
loss: 0.673876  [   32/  135]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.542615 

Epoch 13
-------------------------------
loss: 0.654589  [   32/  135]
Test Error: 
 Accuracy: 80.0%, Avg loss: 0.475310 

Epoch 14
-------------------------------
loss: 0.633316  [   32/  135]
Test Error: 
 Accur